## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
from citipy import citipy

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# File to Load 
city_data_df = os.path.join("..","Weather_Database","WeatherPy_Database.csv")

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Ginir,ET,7.1333,40.7000,58.19,70,33,5.06,scattered clouds
1,1,Albany,US,42.6001,-73.9662,57.09,45,0,1.43,clear sky
2,2,Houma,US,29.5958,-90.7195,68.00,53,1,9.22,clear sky
3,3,Khatanga,RU,71.9667,102.5000,21.99,94,100,14.83,overcast clouds
4,4,Victoria,HK,22.2855,114.1577,72.32,91,98,1.01,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_user_tem_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp)&(city_data_df["Max Temp"] <= max_temp)]
city_data_user_tem_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Houma,US,29.5958,-90.7195,68.00,53,1,9.22,clear sky
4,4,Victoria,HK,22.2855,114.1577,72.32,91,98,1.01,light rain
8,8,Faanui,PF,-16.4833,-151.7500,79.72,72,19,6.89,light rain
15,15,Hilo,US,19.7297,-155.0900,75.11,83,75,4.61,broken clouds
16,16,Castro,BR,-24.7911,-50.0119,72.52,77,58,3.06,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
city_data_user_tem_df.notnull().sum()

City_ID                336
City                   336
Country                334
Lat                    336
Lng                    336
Max Temp               336
Humidity               336
Cloudiness             336
Wind Speed             336
Weather Description    336
dtype: int64

In [7]:
city_data_user_tem_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_city_data_df = city_data_user_tem_df.dropna()
new_city_data_df.count()

City_ID                334
City                   334
Country                334
Lat                    334
Lng                    334
Max Temp               334
Humidity               334
Cloudiness             334
Wind Speed             334
Weather Description    334
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_city_data_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()


In [10]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Houma,US,68.00,clear sky,29.5958,-90.7195,
4,Victoria,HK,72.32,light rain,22.2855,114.1577,
8,Faanui,PF,79.72,light rain,-16.4833,-151.7500,
15,Hilo,US,75.11,broken clouds,19.7297,-155.0900,
16,Castro,BR,72.52,broken clouds,-24.7911,-50.0119,
21,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
22,Rikitea,PF,75.27,light rain,-23.1203,-134.9692,
26,Limbang,MY,77.11,light rain,4.7500,115.0000,
33,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,
35,Atuona,PF,77.76,clear sky,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Houma,US,68.00,clear sky,29.5958,-90.7195,Courtyard by Marriott Houma
4,Victoria,HK,72.32,light rain,22.2855,114.1577,Mini Hotel Central
8,Faanui,PF,79.72,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
15,Hilo,US,75.11,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
16,Castro,BR,72.52,broken clouds,-24.7911,-50.0119,CHACARA BAILLY


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Houma,US,68.00,clear sky,29.5958,-90.7195,Courtyard by Marriott Houma
4,Victoria,HK,72.32,light rain,22.2855,114.1577,Mini Hotel Central
8,Faanui,PF,79.72,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
15,Hilo,US,75.11,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
16,Castro,BR,72.52,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
...,...,...,...,...,...,...,...
694,Jibuti,DJ,78.22,few clouds,11.5000,42.5000,Orchid Main Camp Goubet
695,Acarau,BR,84.81,broken clouds,-2.8856,-40.1200,castelo pizzaria
696,Quatre Cocos,MU,76.08,light rain,-20.2078,57.7625,Sunrise Attitude Hotel
697,Sur,OM,76.28,clear sky,22.5667,59.5289,Sur Plaza Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and{Max Temp} °F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations =  hotel_df[["Lat", "Lng"]]
max_temp =  hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [17]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))